In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

In [1]:
import pyarrow as pa
print(pa.__version__)

14.0.1


In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
test.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


In [5]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [6]:
sample_submission.head()

,id,Fertilizer Name
0,750000,14-35-14 10-26-26 Urea
1,750001,14-35-14 10-26-26 Urea
2,750002,14-35-14 10-26-26 Urea
3,750003,14-35-14 10-26-26 Urea
4,750004,14-35-14 10-26-26 Urea


In [7]:
len(test)

250000

In [8]:
len(train)

750000

In [9]:
len(sample_submission)

250000

In [10]:
train['Fertilizer Name'].value_counts()

Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

## Use the most common Fertilizers as the predcitions

In [11]:
sample_submission['Fertilizer Name']= '14-35-14 10-26-26 17-17-17'
sample_submission.to_csv('most_common.csv',index=False)

# EDA on the data

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           250000 non-null  int64 
 1   Temparature  250000 non-null  int64 
 2   Humidity     250000 non-null  int64 
 3   Moisture     250000 non-null  int64 
 4   Soil Type    250000 non-null  object
 5   Crop Type    250000 non-null  object
 6   Nitrogen     250000 non-null  int64 
 7   Potassium    250000 non-null  int64 
 8   Phosphorous  250000 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.2+ MB


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB


In [14]:
test.describe()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.00000
mean,874999.500000,31.491648,61.045120,45.190444,23.139612,9.487764,21.12206
std,72168.927986,4.024093,6.636372,11.793167,11.215956,5.766860,12.38087
min,750000.000000,25.000000,50.000000,25.000000,4.000000,0.000000,0.00000
25%,812499.750000,28.000000,55.000000,35.000000,13.000000,4.000000,10.00000
50%,874999.500000,31.000000,61.000000,45.000000,23.000000,10.000000,21.00000
75%,937499.250000,35.000000,67.000000,55.000000,33.000000,14.000000,32.00000
max,999999.000000,38.000000,72.000000,65.000000,42.000000,19.000000,42.00000


In [15]:
train.describe()

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,31.503565,61.038912,45.184147,23.093808,9.478296,21.073227
std,216506.495284,4.025574,6.647695,11.794594,11.216125,5.765622,12.346831
min,0.000000,25.000000,50.000000,25.000000,4.000000,0.000000,0.000000
25%,187499.750000,28.000000,55.000000,35.000000,13.000000,4.000000,10.000000
50%,374999.500000,32.000000,61.000000,45.000000,23.000000,9.000000,21.000000
75%,562499.250000,35.000000,67.000000,55.000000,33.000000,14.000000,32.000000
max,749999.000000,38.000000,72.000000,65.000000,42.000000,19.000000,42.000000


In [16]:
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [17]:
test['Soil Type'].value_counts(normalize=True)

Soil Type
Sandy     0.209292
Black     0.200588
Clayey    0.199168
Red       0.197436
Loamy     0.193516
Name: proportion, dtype: float64

In [18]:
train['Soil Type'].value_counts(normalize=True)

Soil Type
Sandy     0.208947
Black     0.201275
Clayey    0.197843
Red       0.197469
Loamy     0.194467
Name: proportion, dtype: float64

In [19]:
test['Crop Type'].value_counts(normalize=True)

Crop Type
Paddy          0.113328
Pulses         0.104884
Cotton         0.092364
Tobacco        0.090912
Wheat          0.088376
Millets        0.087556
Barley         0.086864
Sugarcane      0.086268
Oil seeds      0.086108
Maize          0.083612
Ground Nuts    0.079728
Name: proportion, dtype: float64

In [20]:
train['Crop Type'].value_counts(normalize=True)

Crop Type
Paddy          0.114339
Pulses         0.104240
Cotton         0.092228
Tobacco        0.090667
Wheat          0.088837
Millets        0.087055
Barley         0.086727
Sugarcane      0.085692
Oil seeds      0.085579
Maize          0.083425
Ground Nuts    0.081212
Name: proportion, dtype: float64

## Findings

1. There are no null values in the data.
2. The distributions of the test and train datasets appear to be similar when examining each variable individually.

# Initial XGB model

Here I'm trying the basic XGB model based on this code
https://www.kaggle.com/code/lizzylee1111111111/single-xgboost-baseline

In [28]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [29]:
cat_cols = [col for col in train.select_dtypes(include=['object', 'category']).columns 
            if col != "Fertilizer Name"]

for i in cat_cols:
    label_enc = LabelEncoder()
    train[i] = label_enc.fit_transform(train[i])
    test[i] = label_enc.transform(test[i])

fer_label_enc = LabelEncoder()
train["Fertilizer Name"] = fer_label_enc.fit_transform(train["Fertilizer Name"])

for col in cat_cols:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")


X = train.drop(columns=["id",  'Temparature', 'Humidity', 'Moisture', 'Soil Type', "Fertilizer Name"])
y = train["Fertilizer Name"]
X_test = test.drop(columns=["id", 'Temparature', 'Humidity', 'Moisture', 'Soil Type'])



In [30]:
test.columns

Index(['id', 'Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous'],
      dtype='object')

In [31]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

FOLDS = 5
#skf = KFold(n_splits=FOLDS, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof = np.zeros(shape = (len(train) ,y.nunique()))
pred_prob = np.zeros(shape = (len(test),y.nunique()))

xgb_model = XGBClassifier(
    max_depth=12,
    colsample_bytree=0.467,
    subsample=0.86,
    n_estimators=4000,
    learning_rate=0.03,
    gamma=0.26,
    max_delta_step=4,
    reg_alpha=2.7,
    reg_lambda=1.4,
    early_stopping_rounds=100,
    objective='multi:softprob',
    random_state=13,
    enable_categorical=True,
    tree_method='hist'      
)

for i, (train_idx, valid_idx) in enumerate(skf.split(X,y)):
    print('#' * 15, i+1, '#' *15)
    x_train, x_valid = X.iloc[train_idx],X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx],y.iloc[valid_idx]

    xgb_model.fit(x_train,y_train, eval_set=[(x_valid,y_valid)],verbose = 0)
    oof[valid_idx] = xgb_model.predict_proba(x_valid)
    pred_prob +=xgb_model.predict_proba(X_test)

    top_3_preds = np.argsort(oof[valid_idx], axis=1)[:, -3:][:, ::-1]  
    actual = [[label] for label in y_valid]
    map3_score = mapk(actual, top_3_preds)
    print(f"FOLD {i+1}: MAP@3 Score: {map3_score:.5f}")

############### 1 ###############
FOLD 1: MAP@3 Score: 0.30312
############### 2 ###############
FOLD 2: MAP@3 Score: 0.30287
############### 3 ###############
FOLD 3: MAP@3 Score: 0.30279
############### 4 ###############
FOLD 4: MAP@3 Score: 0.30254
############### 5 ###############
FOLD 5: MAP@3 Score: 0.30373


In [42]:
top_3_preds = np.argsort(pred_prob, axis=1)[:, -3:][:, ::-1]
top_3_labels = fer_label_enc.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)

submission = pd.DataFrame({
    'id': sample_submission['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('submission_xgb_initial.csv', index=False)


In [40]:
len(train['Fertilizer Name'].unique())

7

In [ ]:
# 